# TradingBot

### IMPORTS + EINRICHTUNG GOOGLEDRIVE
> Hier muss ein Google-Konto veknüpft werden, so dass Anweisungen und der Log auf der Drive eingerichtet und gespeichert werden können.

In [88]:
# IMPORTS
from datetime import date
from datetime import datetime
from datetime import timedelta

from google.colab import drive

import csv
import random

drive.mount('/content/drive')

!mkdir /content/drive/My\ Drive/BigData/Anweisungen
!mkdir /content/drive/My\ Drive/BigData/Users
!mkdir /content/drive/My\ Drive/BigData/Logs
!mkdir /content/drive/My\ Drive/BigData/Actions


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/My Drive/BigData/Anweisungen’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/BigData/Users’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/BigData/Logs’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/BigData/Actions’: File exists


### PERSISTENCYCONTROLLER

> Dient zum Speichern und Verwalten der User des Bots. Aktuell wird hier jedoch nur der Action-Log verwaltet.\
> Pfad: ./BigData/Actions/u-[user_id]-[date].csv\
> Hier werden alle täglichen Actions gespeichert.

### LOGGER

> Erstellt die nötigen Strings für das Abspeichern und führt täglichen Speicherbefehl aus.

In [0]:
class PersistencyController:
  def __init__(self, main_folder, user):
    self.user = user
    self.main_folder = main_folder
    self.header = "action_id,action,action_amount,action_course,action_time\n"
    self.log = []

  def saveUser(self, user):
    today = datetime.today()
    path = self.main_folder + 'users/u-' + str(user.user_id) + '-' + str(today)[0:10] + '.csv'

  def saveLog(self, user, log):
    today = datetime.today()
    path = self.main_folder + 'logs/l-' + str(user.user_id) + '-' + str(today)[0:10] + '.csv'

  def addLogInArray(self, log):
    self.log.append(log)
  
  def addLog(self, day):
    path = self.main_folder + "Actions/u-" + str(self.user.user_id) + '-' + str(day)[0:10] + '.csv'
    new_log = self.header

    for l in self.log: 
      new_log += l + "\n"

    with open(path, 'w') as newLog:
      newLog.write(new_log)

    self.log = []


class Logger:
  def __init__(self, persistence):
    self.count = 0
    self.persistence = persistence

  def addLog(self, action):
    log = str(self.count) + "," + action 
    self.count += 1
    self.persistence.addLogInArray(log)

  def saveLog(self, day):
    self.persistence.addLog(day)
    self.count = 0

In [0]:
class Offer:
  def __init__(self, offer_id, offer_object, offer_course, offer_amount):
    self.offer_id = offer_id
    self.offer_object = offer_object
    self.offer_course = offer_course
    self.offer_amount = offer_amount

  def __str__(self):
    return "" + str(self.offer_id) + " :::: (" + self.offer_object + ") course: " + str(self.offer_course) + " - " + str(self.offer_amount)

class User:
  def __init__(self, user_id, user_name, user_budget, user_portfolio):
    self.user_id = user_id
    self.user_name = user_name
    self.user_budget = user_budget
    self.user_portfolio = user_portfolio

## API CLIENT

> Soll der Client in einer weiteren Testphase mit Handelsmärkten 
> zusammenarbeiten, muss hier die Schnittstelle zu diesen 
> eingerichtet werden.\
> Aktuell wird nur ein print-Statement ausgegeben

In [0]:
class FakeApiClient:
  def __init__(self):
    print(":: FakeApiClient is ready ::")

  def sendBuyOffer(self, offer):
    print(":: Sending Offer from BuyHelper ::")

  def sendSellOffer(self, trade):
    print(":: Sending Offer from SellHelper ::")

In [0]:
class Signal:
  def __init__(self, signal_id, signal_currency, signal_course, signal_sign, signal_strength):
    self.signal_id = signal_id
    self.signal_currency = signal_currency
    self.signal_course = signal_course
    self.signal_sign = signal_sign
    self.signal_strength = signal_strength
    self.setSignalAmount(signal_sign, signal_strength)
  
  def setSignalAmount(self, sign, strength):
    self.signal_amount = -1
    if (sign == 'K'):
      if (strength == '0'):
        self.signal_amount = 0.005
      elif (strength == '1'):
        self.signal_amount = 0.01
      elif (strength == '2'):
        self.signal_amount = 0.025
      elif (strength == '3'):
        self.signal_amount = 0.04
    else:
      if (strength == '0'):
        self.signal_amount = 0.15
      elif (strength == '1'):
        self.signal_amount = 0.33
      elif (strength == '2'):
        self.signal_amount = 0.66
      elif (strength == '3'):
        self.signal_amount = 1.0

class SignalLoader:
  def __init__(self):
    print("SignalLoder is ready.")

  def getSignals(self, day):
    _signals = []
    with open('/content/drive/My Drive/BigData/Anweisungen/' + str(day)[0:10] + '.csv', 'r') as signals:
      reader = csv.DictReader(signals)

      for row in reader:
        _signal = Signal(row['signal_id'], row['signal_currency'], row['signal_course'], row['signal_sign'], row['signal_strength'])
        _signals.append(_signal)
      
      return _signals


## BUDGETCLIENT

> Verwaltet das Guthaben des angemeldeten Nutzers und führt Logging aus.

In [0]:
class BudgetClient:
  def __init__(self, trading_user, persistence):
    self.logger = Logger(persistence)
    self.trading_user = trading_user
    print(":: BudgetClient is ready ::")

  def hasValue(self):
    if (self.trading_user.user_budget > 10.0):
      return True
    else:
      return False

  def didBuy(self, offer):
    now = datetime.now()
    amount = offer.offer_amount
    course = offer.offer_course

    self.trading_user.user_budget -= amount

    log_str = "buy," + str(amount) + "," + str(course) + "," + now.strftime('%H:%M:%S')

    self.logger.addLog(log_str)

  def didSell(self, offer):
    now = datetime.now()
    amount = offer.offer_amount
    course = offer.offer_course

    self.trading_user.user_budget += amount

    log_str = "sell," + str(amount) + "," + str(course) + "," + now.strftime('%H:%M:%S')

    self.logger.addLog(log_str)
    

class PortfolioClient:
  def __init__(self, trading_user):
    self.trading_user = trading_user
    print(":: PortfolioClient is ready ::")
  
  def portfolioNotEmpty(self):
    if (len(self.trading_user.user_portfolio) == 0):
      return False
    return True

  def addOffer(self, offer: Offer):
    self.trading_user.user_portfolio.append(offer)
  
  def removeOfferWithId(self, offer_id):
    newPortfolio = []
    oldPortfolio = self.trading_user.user_portfolio

    for offer in oldPortfolio:
      if (offer.offer_id != offer_id):
        newPortfolio.append(offer)
    
    self.trading_user.user_portfolio = newPortfolio
  
  def updateOfferWithId(self, offer_id, signal_amount):
    oldPortfolio = self.trading_user.user_portfolio

    for offer in oldPortfolio:
      amout = -1
      if (offer.offer_id == offer_id):
        if (offer.offer_amount - signal_amount > 0):
          amount = offer.offer_amount-signal_amount
        else:
          amount = 0
        newOffer = Offer(offer.offer_id, offer.offer_object, offer.offer_course, amount)
        return newOffer

  def updateOffers(self, offers):
    self.trading_user.user_portfolio = offers

In [0]:
class BuyHelper:
  def __init__(self, bc: BudgetClient):
    self.bc = bc
    print(":: BuyHelper is ready ::")

  def createBuyOffer(self, api_client: FakeApiClient, signal: Signal):
    print(":: Create Buy Offer ::")

    buy_trade = {
        "id": signal.signal_id,
        "trade_currency": "BTC",
        "trade_object": signal.signal_currency,
        "trade_course": signal.signal_course,
        "trade_amount": signal.signal_amount,
    }

    offer_id = -1
    offer_signal = "K"
    offer_object = ""
    offer_course = -1
    offer_amount = -1

    for key in buy_trade:
      if (key == "id"):
        offer_id = buy_trade[key]
      elif (key == "trade_object"):
        offer_object = buy_trade[key]
      elif (key == "trade_course"):
        offer_course = buy_trade[key]
      elif (key == "trade_amount"):
        offer_amount = buy_trade[key]
    
    offer = Offer(offer_id, offer_object, offer_course, offer_amount)

    self.bc.didBuy(offer)

    api_client.sendBuyOffer(offer)

    return offer

In [0]:
class SellHelper:
  def __init__(self, bc: BudgetClient):
    self.bc = bc
    print(":: SellHelper is ready ::")

  def checkSells(self, offer: Offer, signal: Signal):
    if (offer.offer_object == signal.signal_currency):
      return True
    return False

  def createSellOffer(self, api_client: FakeApiClient, signal: Signal):
    print(":: Create Sell Offer ::")

    sell_trade = {
        "id": signal.signal_id,
        "trade_currency": "BTC",
        "trade_object": signal.signal_currency,
        "trade_course": signal.signal_course,
        "trade_amount": signal.signal_amount,
    }

    offer_id = -1
    offer_signal = "V"
    offer_object = ""
    offer_course = -1
    offer_amount = -1

    for key in sell_trade:
      if (key == "id"):
        offer_id = sell_trade[key]
      elif (key == "trade_object"):
        offer_object = sell_trade[key]
      elif (key == "trade_course"):
        offer_course = sell_trade[key]
      elif (key == "trade_amount"):
        offer_amount = sell_trade[key]
    
    offer = Offer(offer_id, offer_object, offer_course, offer_amount)

    api_client.sendSellOffer(offer)

    self.bc.didSell(offer)

    return offer

## MOCKCLIENT

> Erstellt beim starten Random Signal-Daten, die anschließend im Bot verwendet werden.

In [0]:
class MockClient:
  
  def __init__(self):
    self.main_drive = '/content/drive/My Drive/BigData/'
    self.coins = ["ABC", "BCD", "CDE", "DEF"]
    self.signals = ["V", "K", "H"]
    self.strength = [0, 1, 2, 3]
    self.course_abc = [43.212, 42.112, 46.212]
    self.course_bcd = [12.123, 13.234, 14.345]
    self.course_cde = [8.456, 9.567, 10.678]
    self.course_def = [2.222, 2.333, 2.444]

  def createSignalsForDay(self, day):
    with open(self.main_drive + 'Anweisungen/' + str(day)[0:10] + '.csv', 'w') as newSignal:
      signal_string = "signal_id,signal_sign,signal_strength,signal_course,signal_currency\n"
      index_array = []
      for i in range(0,2):
        signal_index = random.randint(0, 3)
        v_k_h_index = random.randint(0,2)
        if (i == 0):
          index_array.append(signal_index)
          signal_coin = self.coins[signal_index]

          strength_index = random.randint(0, 3)
          signal_strength = self.strength[strength_index]

          if (signal_index == 0):
            signal_courses = self.course_abc
          elif (signal_index == 1):
            signal_courses = self.course_bcd
          elif (signal_index == 2):
            signal_courses = self.course_cde
          else:
            signal_courses = self.course_def

          course_index = random.randint(0, 2)
          signal_course = signal_courses[course_index]

          signal_string += str(random.randint(0, 1234567)) + "," + self.signals[v_k_h_index] + "," + str(signal_strength) + "," + str(signal_course) + "," + str(signal_coin) + '\n'
        else:
          for index in index_array:
            if (index == signal_index):
              return
          
          index_array.append(signal_index)
          signal_coin = self.coins[signal_index]

          strength_index = random.randint(0, 3)
          signal_strength = self.strength[strength_index]

          if (signal_index == 0):
            signal_courses = self.course_abc
          elif (signal_index == 1):
            signal_courses = self.course_bcd
          elif (signal_index == 2):
            signal_courses = self.course_cde
          else:
            signal_courses = self.course_def

          course_index = random.randint(0, 2)
          signal_course = signal_courses[course_index]

          signal_string += str(random.randint(0, 1234567)) + "," + self.signals[v_k_h_index] + "," + str(signal_strength) + "," + str(signal_course) + "," + str(signal_coin) + '\n'

      newSignal.write(signal_string)


In [0]:
class TradingBot:
  def __init__(self, user: User):
    print(":: Init TradingBot ::")
    self.trade_user = user
    self.portfolio_client = PortfolioClient(user)
    self.signal_loader = SignalLoader()
    self.fake_api_client = FakeApiClient()
    self.persistency_client = PersistencyController('/content/drive/My Drive/BigData/', user)
    self.budget_client = BudgetClient(user, self.persistency_client)
    self.buy_helper = BuyHelper(self.budget_client)
    self.sell_helper = SellHelper(self.budget_client)
    print(":: TradingBot is ready ::")

  def start(self, day):
    mock_client = MockClient()
    if (day == ""):
      today = datetime.today()
    else: 
      today = day

    for d in range(0,31):
      newDay = today + timedelta(days=d)
      mock_client.createSignalsForDay(newDay)
    
    self.start_date = today

  def loadSignals(self, day):
    return self.signal_loader.getSignals(day)

  def run31Days(self, day):

    if (day == ""):
      print()
      for d in range(0, 31):
        print(":: DAY #" + str(d) + " ::")
        today = self.start_date + timedelta(days=d)

        _signals = self.loadSignals(today)

        _kaufen = []
        _verkaufen = []
        _halten = []

        for sig in _signals:
          if (sig.signal_sign == "V"):
            _verkaufen.append(sig)
          elif (sig.signal_sign == "K"):
            _kaufen.append(sig)
          elif (sig.signal_sign == "H"):
            _halten.append(sig)

        self.handleVerkaufen(_verkaufen)
        self.handleKaufen(_kaufen)
        self.handleHalten(_halten)

        print()
        print(":: END OF DAY ::")
        print()
        for p in user.user_portfolio:
          print(p)
        print()
        print(":: :: :: :: :: ::")
        print()
        self.budget_client.logger.saveLog(today)

  def handleVerkaufen(self, signale):
    for s in signale:
      self.sell(s)

  def handleKaufen(self, signale):
    for s in signale:
      self.buy(s)
  
  def handleHalten(self, signale):
    for s in signale:
      print(":: halten ::")

  def buy(self, buy_signal):
    if (self.budget_client.hasValue()):
      buy = self.buy_helper.createBuyOffer(self.fake_api_client, buy_signal)
      self.portfolio_client.addOffer(buy)
    else:
      print("not enoug money.")

  def sell(self, signal):
    offers = []
    for offer in self.trade_user.user_portfolio:
      if (self.sell_helper.checkSells(offer, signal)):
        sellOffer = self.sell_helper.createSellOffer(self.fake_api_client, signal)
        if (signal.signal_amount == 1.0):
          self.portfolio_client.removeOfferWithId(offer.offer_id)
        else:
          newOffer = self.portfolio_client.updateOfferWithId(offer.offer_id, signal.signal_amount)
          offers.append(newOffer)
    
    self.portfolio_client.updateOffers(offers)

  def save(self):
    self.persistency_client.saveUser(self.trade_user)
    self.persistency_client.saveLog(self.trade_user, "abc")

In [100]:
user = User(1, "alexpuchta", 100.00, [])
bot = TradingBot(user)

bot.start("")

bot.run31Days("")


:: Init TradingBot ::
:: PortfolioClient is ready ::
SignalLoder is ready.
:: FakeApiClient is ready ::
:: BudgetClient is ready ::
:: BuyHelper is ready ::
:: SellHelper is ready ::
:: TradingBot is ready ::

:: DAY #0 ::

:: END OF DAY ::


:: :: :: :: :: ::

:: DAY #1 ::
:: Create Buy Offer ::
:: Sending Offer from BuyHelper ::

:: END OF DAY ::

1002443 :::: (BCD) course: 14.345 - 0.005

:: :: :: :: :: ::

:: DAY #2 ::

:: END OF DAY ::

1002443 :::: (BCD) course: 14.345 - 0.005

:: :: :: :: :: ::

:: DAY #3 ::

:: END OF DAY ::


:: :: :: :: :: ::

:: DAY #4 ::
:: halten ::

:: END OF DAY ::


:: :: :: :: :: ::

:: DAY #5 ::
:: Create Buy Offer ::
:: Sending Offer from BuyHelper ::

:: END OF DAY ::

1188100 :::: (CDE) course: 9.567 - 0.01

:: :: :: :: :: ::

:: DAY #6 ::

:: END OF DAY ::

1188100 :::: (CDE) course: 9.567 - 0.01

:: :: :: :: :: ::

:: DAY #7 ::
:: halten ::

:: END OF DAY ::


:: :: :: :: :: ::

:: DAY #8 ::
:: Create Buy Offer ::
:: Sending Offer from BuyHelper 